# Introduction

**Contents:** Introduces the `EconModelClass`

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from EconModel import EconModelClass, jit  

# EconModel

**Docunmentation:** [EconModel](https://github.com/NumEconCopenhagen/EconModel) github repository

**Examples:** [EconModelNotebooks](https://github.com/NumEconCopenhagen/EconModelNotebooks) github repository

**Overview:** `EconModel` is a small package for easily working with economic models in Python. It has four objectives:

1. Make it easy to write well-structed code.
1. Provide standard functionality for copying, saving and loading.
1. Provide an easy interface to call [numba](http://numba.pydata.org/) JIT compilled functions.
1. Provide an easy interface to call C++ functions [*not relevant in this course*].

The two first objectives should be of interest to everybody, while the two last objectives are for more advanced users.

## Baseline Usage

The model is **required** to have the following three methods:

1. `.settings()`: Choose fundamental settings.
1. `.setup()`: Set free parameters.
1. `.allocate()`: Set compound parameters and allocate arrays.

When the model is initialized `.settings()`, `.setup()` and `.allocate()` are all called (in that order).

Per default the namespaces `.par` (for parameters), `.sol` (for solution objects) and `.sim` (for simulation objects) are available. The philosophy of the package is that all the content of the model is contained in these namespaces. After the model has been initialized, no additional entries should be added to the namespaces, and each entry should only change *value(s)* (or shape for arrays), but never *type* (or number of dimensions for arrays).

In [2]:
class MyModelClass(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
            
        pass # nothing chosen here
    
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        sol = self.sol
        
        par.X = np.linspace(0,10,par.N)
        sol.Y = np.zeros(par.N)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        sol = self.sol
        
        sol.Y[:] = par.X*(par.a+par.b)


In `.settings()` the user can change the default behavior by specifying: 

1. `self.savefolder = str`: Filepath to save in and load from (default: 'saved').
1. `self.namespaces = [str]`: List of namespaces (in addition to `.par`, `.sol`, `.sol`).
1. `self.other_attrs = [str]`: List of additional attributes to be copied and saved.


### Setup

In [3]:
model = MyModelClass(name='example')

In [4]:
model.solve()

**Print description:**

In [5]:
print(model)

Modelclass: MyModelClass
Name: example

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sol:
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



**Updating at initialization:**

In [6]:
model_alt = MyModelClass(name='alt',par={'a':3.0,'b':1.2})
print(model_alt.par.a)

3.0


**Unpack for later use:**

In [7]:
par = model.par
sol = model.sol

**Under the hood:** Each namespace is techically a `SimpleNamespace` (from `types`)

In [8]:
type(par)

types.SimpleNamespace

### Copy, save and load

In [9]:
model_copy = model.copy(name='example_copy')
model_copy.save()

In [10]:
model_loaded = MyModelClass(name='example_copy',load=True)
print(model_loaded)

Modelclass: MyModelClass
Name: example_copy

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sol:
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



### To and from dictionary 

In [11]:
modeldict = model.as_dict()
model_dict = MyModelClass(name='example_dict',from_dict=modeldict)
print(model_dict)

Modelclass: MyModelClass
Name: example_dict

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sol:
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



## Numba

**Goal:** Call [numba](http://numba.pydata.org/) JIT compilled functions easily using namespaces.

**Problem:** [numba](http://numba.pydata.org/) only allows specific types (and not e.g. `SimpleNamespace`).

**Under-the-hood:** `jit(model)` temporarily turns all namespaces into `namedtuples`, which can be used as input in JIT compilled functions. 

In [12]:
@nb.njit
def fun(par,sol):
    sol.Y[:] = par.X*(par.a+par.b)

In [13]:
def test_numba(model):
    with jit(model) as model_jit:
        fun(model_jit.par,model_jit.sol)

In [14]:
test_numba(model)

In [15]:
%time test_numba(model)
%time test_numba(model)

Wall time: 1e+03 µs
Wall time: 0 ns


**Note:** The first run is slower due to compiling.

**Check result:**

In [16]:
assert np.allclose(par.X*(par.a+par.b),sol.Y)

### Advanced: Recompilling

When a new model is created, re-compilation of JIT functions are normally necessary:

In [17]:
model_new = MyModelClass(name='numba_new')
%time test_numba(model_new)
%time test_numba(model_new)

Wall time: 186 ms
Wall time: 0 ns


This is *not* the case, when a copy is made instead:

In [18]:
model_new_ = model_new.copy(name='numba_new_copy')
%time test_numba(model_new_)

Wall time: 0 ns


### Advanced: Type inference

When the model is created **all types are infered** and the information is saved in the nested-dict `_ns_specs`. E.g.:

In [19]:
model._ns_specs['types']

{'par': {'N': [int, numpy.int32],
  'a': [float, numpy.float64],
  'b': [float, numpy.float64],
  'threads': [int, numpy.int32],
  'txt': [str],
  'txtlist': [str],
  'X': [numpy.ndarray]},
 'sol': {'Y': [numpy.ndarray]},
 'sim': {}}

In [20]:
model._ns_specs['np_dtypes']

{'par': {'X': dtype('float64')}, 'sol': {'Y': dtype('float64')}, 'sim': {}}

In [21]:
model._ns_specs['np_ndims']

{'par': {'X': 1}, 'sol': {'Y': 1}, 'sim': {}}

**The types must NOT change afterwards**. If they do, an error is raised when using  `jit`.

In [22]:
def test_numba_errors(model):
    try:
        test_numba(model)
    except ValueError as e:
        print(f'ValueError: {e}')
    else:
        print('no errors raised')

In [23]:
model_test = model.copy()
model_test.par.a = 1
test_numba_errors(model_test)

ValueError: par.a has type <class 'int'>, should be in [<class 'float'>, <class 'numpy.float64'>]


In [24]:
model_test = model.copy()
model_test.par.c = np.nan
test_numba_errors(model_test)

ValueError: c is not allowed in .par


In [25]:
model_test = model.copy()
model_test.par.X = np.zeros((1,1))
test_numba_errors(model_test)

ValueError: par.X has ndim 2, should be 1


In [26]:
model_test = model.copy()
model_test.par.X = model.par.X.astype('int')
test_numba_errors(model_test)

ValueError: par.X has dtype int32, should be float64


**If types *must* to be changed, then it is necessary to re-infer types:**

In [27]:
model_test = model.copy()
model_test.par.a = 1
model_test.infer_types()
test_numba_errors(model_test)
model_test.par.a = 1.0
test_numba_errors(model_test)

no errors raised
ValueError: par.a has type <class 'float'>, should be in [<class 'int'>, <class 'numpy.int32'>]


**Namedtuples** are immutables, so attributes cannot be changed while using `jit`:

In [28]:
try:
    with jit(model,show_exc=True) as model_jit:
        model_jit.par.a = 3.0
except AttributeError as e:
    pass

Traceback (most recent call last):
  File "C:\Users\gmf123\AppData\Local\Temp\ipykernel_6620\2942967087.py", line 3, in <cell line: 1>
    model_jit.par.a = 3.0
AttributeError: can't set attribute


## In-class exercise

1. Look at the notebook above and talk about it with the person next to you for 3 minutes
2. Write at least one question on https://padlet.com/jeppe_druedahl/advmacrohet